In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [10]:
df = pd.read_csv('city-state_zip-codes.csv')
df

,City-State,Zip_Code
0,Clarksburg-MD,['20871']
1,Boyds-MD,['20841']
2,Rockville-MD,"['20850', '20851', '20852', '20853']"
3,Germantown-MD,"['20874', '20876']"
4,Gaithersburg-MD,"['20877', '20878', '20879', '20882']"
5,Frederick-MD,"['21701', '21702', '21703', '21704']"
6,Ashburn-VA,"['20147', '20148']"
7,Beaumont-VA,['23014']
8,Sterling-VA,"['20164', '20165', '20166']"


In [3]:
df.columns

Index(['City-State', 'Zip_Code'], dtype='object')

In [9]:
def trim_string(word):
    """Removes the first and last characters from a string."""
    if len(word) <= 2:
        return ''  # Return an empty string if word is too short
    return word[1:-1]

In [11]:
# Removing brackets by joining the values into a single string
df['Zip_Code'] = df['Zip_Code'].apply(lambda x: trim_string(x))
df

,City-State,Zip_Code
0,Clarksburg-MD,'20871'
1,Boyds-MD,'20841'
2,Rockville-MD,"'20850', '20851', '20852', '20853'"
3,Germantown-MD,"'20874', '20876'"
4,Gaithersburg-MD,"'20877', '20878', '20879', '20882'"
5,Frederick-MD,"'21701', '21702', '21703', '21704'"
6,Ashburn-VA,"'20147', '20148'"
7,Beaumont-VA,'23014'
8,Sterling-VA,"'20164', '20165', '20166'"


In [15]:
explode_df = df.assign(Zip_Code=df['Zip_Code'].str.split(",")).explode('Zip_Code') 
explode_df

,City-State,Zip_Code
0,Clarksburg-MD,'20871'
1,Boyds-MD,'20841'
2,Rockville-MD,'20850'
2,Rockville-MD,'20851'
2,Rockville-MD,'20852'
2,Rockville-MD,'20853'
3,Germantown-MD,'20874'
3,Germantown-MD,'20876'
4,Gaithersburg-MD,'20877'
4,Gaithersburg-MD,'20878'
